In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
%matplotlib notebook

In [2]:
file = '../data/spain-red'
df = pd.read_csv(file + '.csv', converters={'foods': literal_eval, 'highlights': literal_eval})

In [3]:
diacritics = {'&#237;': 'í', '&#243;': 'ó', '&#250;': 'ú', '&#241;': 'ñ', '&#232;': 'è', \
    '&#8364;': '€', '&#193;': 'Á', '&#192;': 'À', '&#233;': 'é', '&#224;': 'à', \
    '&#239;': 'ï', '&#231;': 'ç', '&#193;': 'Á', '&#252;': 'ü', '&#225;': 'á', '&#186;': 'º', \
    '&#244;': 'ô', '&#8217;': '’', '&#960;': 'π', '&#210;': 'Ò'}
df.replace(diacritics, regex=True, inplace=True)
df['winery'] = df['winery'].apply(lambda x: re.sub(r'((?<=[a-z_à-ÿ])[A-Z]|(?<!\A)[A-Z](?=[a-z_à-ÿ]))', r' \1', x))
df['vintage'] = df['vintage'].apply(lambda x: re.sub(r'((?<=[a-z_à-ÿ])[A-Z]|(?<!\A)[A-Z](?=[a-z_à-ÿ]))', r' \1', x))
df['region'] = df['region'].apply(lambda x: re.sub(r'((?<=[a-z_à-ÿ])[A-Z]|(?<!\A)[A-Z](?=[a-z_à-ÿ]))', r' \1', x))
df['vintage'] = df['vintage'].apply(lambda x: re.sub(r"(\S)\(", r'\1 (', x))
df['vintage'] = df['vintage'].apply(lambda x: re.sub('\(\s*(.*?)\s*\)', r'(\1)', x))
df['vintage'] = df['vintage'].apply(lambda x: re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", x).strip())
df['vintage'] = df['vintage'].replace(r'\s+', ' ', regex=True)

numeric_columns = ['year', 'rating', 'ratings_count', 'country_rank', 'region_rank', \
                   'winery_rank', 'global_rank', 'alcohol', 'body', 'acidity', 'price']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

df = df.drop(['drink_from', 'drink_until'], axis = 1)

for f in df['foods']:
    f.sort()

In [4]:
df.describe()

,year,rating,ratings_count,country_rank,region_rank,winery_rank,global_rank,alcohol,body,acidity,price
count,2014.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,1656.000000,1900.000000,1900.000000,2013.000000
mean,2014.128600,4.147992,744.478433,3.930590,5.692117,15.629152,4.140803,14.081582,4.271579,2.963158,87.068867
std,6.252022,0.244010,1714.883180,3.456955,5.452801,12.844511,3.426670,1.634316,0.507897,0.188424,234.235746
min,1964.000000,3.700000,25.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,2.000000,4.900000
25%,2013.000000,3.900000,86.000000,1.000000,2.000000,6.000000,1.000000,14.000000,4.000000,3.000000,18.950000
50%,2016.000000,4.100000,227.000000,3.000000,4.000000,12.000000,3.000000,14.500000,4.000000,3.000000,31.130000
75%,2017.000000,4.300000,659.000000,5.000000,8.000000,22.000000,6.000000,14.500000,5.000000,3.000000,61.950000
max,2020.000000,4.900000,32378.000000,22.000000,38.000000,81.000000,20.000000,16.500000,5.000000,3.000000,3404.940000


In [20]:
# Ideal wines: wine under 30 euros, 
# with rating more than median (4.1), 
# number of ratings more than median value (227),
# body of 4–5,
# and  alcohol <= 14.5

ideal_wines = df[(df['price'] <= 30) & (df['rating'] > 4.1) & (df['ratings_count'] > 227) \
                 & (df['body'] >= 4) & (df['alcohol'] <= 14.5)]
ideal_wines.describe()

,year,rating,ratings_count,country_rank,region_rank,winery_rank,global_rank,alcohol,body,acidity,price
count,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000
mean,2015.452381,4.219048,1533.642857,1.071429,2.452381,9.357143,1.619048,14.161905,4.357143,2.976190,23.796429
std,8.311621,0.045468,2704.032750,0.260661,1.549231,7.321273,0.491507,0.487378,0.484966,0.154303,4.784453
min,1964.000000,4.200000,233.000000,1.000000,1.000000,1.000000,1.000000,12.500000,4.000000,2.000000,7.300000
25%,2016.000000,4.200000,443.750000,1.000000,1.000000,2.000000,1.000000,14.000000,4.000000,3.000000,21.610000
50%,2017.000000,4.200000,668.000000,1.000000,2.000000,9.000000,2.000000,14.500000,4.000000,3.000000,25.165000
75%,2018.000000,4.200000,1166.500000,1.000000,3.750000,13.750000,2.000000,14.500000,5.000000,3.000000,27.615000
max,2020.000000,4.400000,16395.000000,2.000000,6.000000,27.000000,2.000000,14.500000,5.000000,3.000000,29.950000


In [21]:
ideal_wines = ideal_wines[['winery', 'vintage', 'year', 'region', 'rating', 'ratings_count', \
             'foods', 'alcohol', 'style', 'body', 'body_desc', 'acidity', \
             'grapes', 'price']]

In [23]:
ideal_wines['foods'] = ideal_wines['foods'].astype(str)
ideal_wines['foods'] = ideal_wines['foods'].apply(lambda x: x.replace('[','').replace(']','').replace("'",''))
ideal_wines['year'] = ideal_wines['year'].astype(int)

,winery,vintage,year,region,rating,ratings_count,foods,alcohol,style,body,body_desc,acidity,grapes,price
17,Abadía Retuerta,Selección Especial,2017.0,Sardónde Duero,4.4,3026,"Beef, Pasta, Poultry, Veal",14.5,Spanish Red,4.0,Full-bodied,3.0,Shiraz/Syrah,26.33
115,Bodegas Amaren,Ángelesde Amaren Tempranillo- Graciano,2015.0,Rioja,4.2,829,"Beef, Lamb, Poultry, Veal",14.0,Spanish Rioja Red,4.0,Full-bodied,3.0,Tempranillo,21.15
261,Bodegas Aragonesas,Aragonia Selección Especial Garnacha,2018.0,Campode Borja,4.2,233,"Beef, Game (deer, venison), Lamb, Pasta, Poultry",14.5,Spanish Grenache,5.0,Very full-bodied,3.0,Garnacha,14.75
270,Arzuaga,Riberadel Duero Crianza,2017.0,Riberadel Duero,4.2,3347,"Beef, Game (deer, venison), Lamb",14.5,Spanish Ribera Del Duero Red,5.0,Very full-bodied,3.0,Tempranillo,24.19
271,Arzuaga,Riberadel Duero Crianza,2018.0,Riberadel Duero,4.3,3483,"Beef, Game (deer, venison), Lamb",14.5,Spanish Ribera Del Duero Red,5.0,Very full-bodied,3.0,Tempranillo,24.75
272,Arzuaga,Riberadel Duero Crianza,2019.0,Riberadel Duero,4.2,763,"Beef, Game (deer, venison), Lamb",14.5,Spanish Ribera Del Duero Red,5.0,Very full-bodied,3.0,Tempranillo,29.50
287,Bodegas Faustino,I Gran Reserva,1964.0,Rioja,4.3,452,"Beef, Lamb, Poultry, Veal",12.5,Spanish Rioja Red,4.0,Full-bodied,3.0,Tempranillo,22.99
411,Carmelo Rodero,Crianza,2017.0,Riberadel Duero,4.2,2044,"Beef, Game (deer, venison), Lamb",14.5,Spanish Ribera Del Duero Red,5.0,Very full-bodied,3.0,Tempranillo,27.89
412,Carmelo Rodero,Crianza,2018.0,Riberadel Duero,4.2,1063,"Beef, Game (deer, venison), Lamb",13.0,Spanish Ribera Del Duero Red,5.0,Very full-bodied,3.0,Tempranillo,25.40
575,Clos Pons,Roc Nu,2011.0,Costersdel Segre,4.2,420,"Beef, Pasta, Poultry, Veal",13.5,Spanish Red,4.0,Full-bodied,3.0,Garnacha,24.95


In [26]:
ideal_wines.to_html('../data/ideal_wines.html', encoding='utf-8', index=False)